In [ ]:
from etsy2 import Etsy
from etsy2.oauth import EtsyOAuthClient, EtsyOAuthHelper
import urllib.parse as urlparse
from urllib.parse import parse_qs

# These should be stored in a .env file
api_key = "" # API key for Etsy API
shared_secret = "" # shared secret for Etsy API
etsy_shop_name = "" # shop to look at
etsy_item = "" # item to search for
# define permission scopes
permission_scopes = ['email_r', 'listings_r', 'transactions_r']

login_url, temp_oauth_token_secret = \
    EtsyOAuthHelper.get_request_url_and_token_secret(api_key, shared_secret, permission_scopes)

query = urlparse.urlparse(login_url).query
temp_oauth_token = parse_qs(query)['oauth_token'][0]

print("Follow this URL:", login_url)

oauth_token, oauth_token_secret = \
    EtsyOAuthHelper.get_oauth_token_via_verifier(api_key, shared_secret, temp_oauth_token, temp_oauth_token_secret, input('Verifier: '))

# OAuth Client
etsy_oauth = EtsyOAuthClient(client_key = api_key,
                             client_secret = shared_secret,
                             resource_owner_key = oauth_token,
                             resource_owner_secret = oauth_token_secret)

# Etsy Connection
etsy = Etsy(etsy_oauth_client = etsy_oauth)

In [46]:
shop = etsy.getShop(shop_id=etsy_shop_name)[0]

In [47]:
shop_id = shop.get('shop_id')

In [ ]:
relevant_transactions = []
num_trans = 0
# hard limit to not exceed API rate limits,
# we can only paginate with 100 results per API call.
for i in range(0, 25000, 100):
    transactions = etsy.findAllShopTransactions(shop_id=shop_id, offset=num_trans, limit=100)
    for transaction in transactions:
        if etsy_item in transaction.get('title'):
            relevant_transactions.append(transaction.get('transaction_id'))
    num_trans += 100
    # print('Transactions scanned:', num_trans)

In [ ]:
print("Found", len(relevant_transactions), "results.")
# print(relevant_transactions)

In [51]:
import time
buyerUserIds = []
buyerUsers = []
buyersNotFound = []
numFetches = 0
for relevant_transaction in relevant_transactions:
    if numFetches % 10 == 0:
        time.sleep(1) # API is limited to 10 calls/sec
    transactionObj = etsy.getShop_Transaction(transaction_id=relevant_transaction)[0]
    buyerUserId = transactionObj['buyer_user_id']
    try:
        buyerUser = etsy.getUser(user_id=buyerUserId)[0]
        buyerUsers.append(buyerUser)
    except:
        buyersNotFound.append(buyerUserId)
        continue
    numFetches += 1

In [ ]:
print(buyerUsers)